# ライフゲーム for sense-hat

#  https://qiita.com/ssh0/items/042b7d0f624290464b7a

In [1]:
import numpy as np
import sys
import time
import os
from sense_hat import SenseHat

class LifeGame:

    def __init__(self, L=8, rule="2 3/3"):
        self.L = L # lattice size
        self.rule = rule
        self.reset()
        
        self.sense = SenseHat()
        self.sense.clear()
        self.C_BB = (0,0,255)
        #self.C_F = (255,255,255)
        self.C_R = (255,    0,     0)
        self.C_G = (     0,255,    0)
        self.C_B = (     0,     0,255)
        self.C_F = self.C_R

        self.survive = [int(i) for i in self.rule.split("/")[0].split()]
        self.birth = [int(i) for i in self.rule.split("/")[1].split()]

        
    def reset(self):
        p = 0.2
        
        lattice = np.random.random([self.L+2, self.L+2])
        self.lattice = lattice<p
        self.lattice[0,:] = self.lattice[self.L+1,:] = False
        self.lattice[:,0] = self.lattice[:,self.L+1] = False
        
        
    def canvas_update(self):
        
        #self.sense.clear()
        for y in range(1,self.L+1):
            for x in range(1,self.L+1):
                if self.lattice[x,y]:
                    #l += u" ■"
                    self.sense.set_pixel(x-1,y-1,self.C_F)
                else:
                    #l += u" □"
                    self.sense.set_pixel(x-1,y-1,self.C_BB)
        
        time.sleep(0.1)

    def progress(self):
        L = self.L
        Tmax = 50
        t = 0
        while t < Tmax:
            try:
                self.canvas_update()
                nextsites = []

                # 周期境界条件
                self.lattice[0,0] = self.lattice[self.L,self.L]
                self.lattice[0,self.L+1] = self.lattice[self.L,1]
                self.lattice[self.L+1,0] = self.lattice[1,self.L]
                self.lattice[self.L+1,self.L+1] = self.lattice[1,1]
                for m in range(1, self.L+1):
                    self.lattice[m, self.L+1] = self.lattice[m, 1]
                    self.lattice[m, 0] = self.lattice[m, self.L]
                    self.lattice[0, m] = self.lattice[self.L, m]
                    self.lattice[self.L+1, m] = self.lattice[1, m]

                # 隣接格子点の判定
                for m in range(1,self.L+1):
                    for n in range(1,self.L+1):

                        if self.lattice[m,n]:
                            neighber = np.sum(self.lattice[m-1:m+2, n-1:n+2])-1
                            if neighber in self.survive:
                                nextsites.append((m,n))
                        else:
                            neighber = np.sum(self.lattice[m-1:m+2, n-1:n+2])
                            if neighber in self.birth:
                                nextsites.append((m,n))

                # latticeの更新
                self.lattice[:] = False
                for nextsite in nextsites:
                    self.lattice[nextsite] = True

                t += 1

            except KeyboardInterrupt:
                #print "stopped."
                break
                

In [ ]:
lg = LifeGame()
while True:
    lg.progress()
    lg.reset()